In [1]:
import glob
import json
import random
import pandas
from pprint import pprint
import sys
import json
import numpy as np
import pymssql
from gemd_database import MSSQLDatabase
from queries import *
import SciServer.CasJobs as cj

In [ ]:
with open('/home/arachid1/.private/gemd_db.json','r') as f:
    AUTH = json.load(f)

DATABASE='GEMD'
GEMDDB=MSSQLDatabase(AUTH,DATABASE)

In [ ]:
sql="""
select distinct *
from GEMDModel
"""
GEMDDB.execute_query(sql)

In [ ]:
sql="""
select top 4 context
  from  gemdobject c where gemd_type='process_run'
 order by newid()
"""
query_results=GEMDDB.execute_query(sql)
query_results
# query_results.iloc[0]['context']

find process runs for a random set of process specs

In [ ]:
sql="""
with some_specs as (  
select top 10 *              -- using SELECT TOP N ... ORDER BY newid() is standard trick for getting a random sample of N entries
  from gemdobject
 where gemd_type='process_spec' 
order by newid()                       
)
select s.uid as spec_id
,      json_value(s.context,'$.name') as spec
,      r.uid as run_id
,      r.context as run
  from some_specs s
  join gemdobject r
 on r.gemd_type='process_run' 
   and JSON_VALUE(r.context,'$.spec.id')=s.uid
order by spec
"""
query_results=GEMDDB.execute_query(sql)
query_results

for a material spec (non empty properties), get material runs and templates

In [ ]:
%%time
sql="""
with some_specs as (
select top 3 *
  from gemdobject
 where gemd_type='material_spec' 
   and context like '%"properties":%'
and context not like '%properties": [[]]%'   -- having some non-trivial properties. NOTE '[' must be escaped to '[[]', ']' needs no escaping
                                             -- see https://docs.microsoft.com/en-us/sql/t-sql/language-elements/like-transact-sql?view=sql-server-ver16#using-wildcard-characters-as-literals
)
select s.uid as spec_id
,      JSON_VALUE(t.context,'$.name') as template
,      JSON_VALUE(s.context,'$.name') as spec
,      JSON_VALUE(m.context,'$.name') as material
,      JSON_VALUE(p.context,'$.name') as process
,      JSON_VALUE(ps.context,'$.name') as process_spec
--,      t.context as t_context
--,      s.context as s_context
--,      m.context as m_context
,      json_query(ps.context,'$.tags') as ps_tags
--,      p.context as p_context
--,      ps.context as ps_context
  from some_specs s
  join gemdobject t on t.uid=json_value(s.context,'$.template.id') -- get template for material_spec
  join gemdobject m on s.uid=json_value(m.context,'$.spec.id')     -- get material_run for material_spec
  join gemdobject p on p.uid=json_value(m.context,'$.process.id')  -- get process_run for material_run
  join gemdobject ps on ps.uid=json_value(p.context,'$.spec.id')   -- get process_spec for process_run
  """
query_results=GEMDDB.execute_query(sql)
query_results.head(4)

get properties for a material spec

In [ ]:
%%time
sql="""
select s.uid as spec_id
,      JSON_VALUE(s.context,'$.name') as spec
,      JSON_VALUE(sp.property,'$.name') as p
,      sp.property
  from gemdobject s
  cross apply OPENJSON(s.context,'$.properties') with (property nvarchar(max) 'lax$.property' as json) sp
 where s.gemd_type='material_spec' 
  """
query_results=GEMDDB.execute_query(sql)
query_results

return reachable nodes from uid node

081bbfd7-1bbe-40d4-bb44-15d78b3e23bb is an uid of Impact Sample Cutting and Polishing from the laser shock model

In [ ]:
query_results=GEMDDB.execute_query(reachable_nodes_query(uid='081bbfd7-1bbe-40d4-bb44-15d78b3e23bb'))
query_results

In [ ]:
query_results=GEMDDB.execute_query(to_node_query(model_id=3))
query_results

In [ ]:
# sql="""
# select distinct *
# from GEMDModel
# """
# ee2196b4-b0eb-4c47-b2ff-4ba3fa58f25e
# GEMDDB.execute_query(sql)

In [ ]:
query_results=GEMDDB.execute_query(multiple_paths_nodes_query(model_id=3))
query_results

In [ ]:
query_results=GEMDDB.execute_query(multiple_paths_nodes_query(model_id=3))
query_results